# RQ (Request vs Question) and TL (Test vs Learn) labeling

The purpose of this notebook is to manually label prompts and add them to the [reddgr/rq-request-question-prompts](https://huggingface.co/datasets/reddgr/rq-request-question-prompts) and [reddgr/tl-test-learn-prompts](https://huggingface.co/datasets/reddgr/tl-test-learn-prompts) datasets.

## Notebook setup

In [1]:
COLAB = False # Set this to True if you want to install the libraries and clone the repository in Colab
USE_DOTENV = True # Set this to False if you don't have a .env file for storing environment variables

Run this cell for cloning the repo on Google Colab or other cloud services

In [2]:
if COLAB:
    USE_DOTENV = False
    dotenv_path = None
    from google.colab import userdata
    colab_secrets = {'HF_TOKEN': userdata.get('HF_TOKEN'), 'HF_TOKEN_WRITE': userdata.get('HF_TOKEN_WRITE')}
    !pip install datasets langdetect
    !git clone https://github.com/reddgr/chatbot-response-scoring-scbn-rqtl
    import os
    os.system("mv chatbot-response-scoring-scbn-rqtl scbn_rqtl")

In [3]:
if USE_DOTENV: 
    COLAB=False
    dotenv_path = "C:/apis/.env"
    colab_secrets = None
if not USE_DOTENV and not COLAB:
    dotenv_path = None
    colab_secrets = None

import torch
from transformers import pipeline
from IPython.display import clear_output
import pandas as pd
from datasets import Dataset
import random
from textwrap import fill

if COLAB:
    from scbn_rqtl import env_options, labeling_widget, text_classification_functions as tcf, lmsys_dataset_handler as lmsys
else:
    import text_classification_functions as tcf
    import labeling_widget
    import env_options
    import lmsys_dataset_handler as lmsys

hf_token, hf_token_write = env_options.check_env(colab=COLAB, use_dotenv=USE_DOTENV, dotenv_path=dotenv_path, colab_secrets=colab_secrets)

Python version: 3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
PyTorch version: 2.2.2
Transformers version: 4.44.2
CUDA device: NVIDIA GeForce RTX 4060 Laptop GPU
Retrieved HuggingFace token(s) from .env file
Using HuggingFace token: hf_M*****************************IASJ
Using HuggingFace write token: hf_u*****************************Xipx


## Importing data from lmsys/lmsys-chat-1m

In [4]:
N_SAMPLES = 500 # Number of full conversations to extract from the dataset: use a high number if streaming (samples chosen at random only if storing locally)
MAX_CHAR_LENGTH = 400 # Maximum character length of the prompts to be labeled

lmsys_chat_1m = lmsys.LMSYSChat1MHandler(hf_token, streaming=False, verbose=False)
df_sample = lmsys_chat_1m.extract_df_sample(N_SAMPLES)
df_prompts = lmsys_chat_1m.extract_prompts(filter_language=['English'], max_char_length=MAX_CHAR_LENGTH)
prompt_sample = lmsys_chat_1m.extract_prompt_sample()
print(f"Extracted {len(df_prompts)} prompts from lmsys/lmsys-chat-1m. Prompt sample:\n")
print(prompt_sample)

Retrieved 500 conversations from lmsys/lmsys-chat-1m
Extracted 596 prompts from lmsys/lmsys-chat-1m. Prompt sample:

Is it true that in 1992, on the set of Baywatch,  NAME_1 was shocked that NAME_2 was physically stronger and a far better athlete than NAME_3 was? NAME_4 was 28-years old, 5'10" tall and weighed 123 pounds. NAME_5 was 18-years old, 5'2" tall and weighed 130 pounds. Could the tiny NAME_5 really be stronger than NAME_4?


Execute the cell below to print random conversations with multiple turns:

In [8]:
# Showing an example of a multi-turn conversation
df_sample_with_turns = lmsys_chat_1m.add_turns_to_conversations()
multi_turn_conversation_indices = df_sample_with_turns[df_sample_with_turns['turn'] > 1].index
random_conversation_index = random.choice(multi_turn_conversation_indices)
print(f"\nConversation ID {df_sample_with_turns.loc[random_conversation_index, 'conversation_id']}:\n")
#print(df_sample_with_turns.loc[random_conversation_index, 'conversation'])
conversation = df_sample_with_turns.loc[random_conversation_index, 'conversation']
for turn in conversation:
    user = turn.get('role')
    content = turn.get('content', '')
    wrapped_content = fill(content, width=120)
    role = '😎' if user == 'user' else '🤖'
    print(f"{role} {wrapped_content}")


Conversation ID 4c7357e146c4436ebc384725db0ac549:

😎 Two people go into a pub.
🤖 Okay, what happens next? Do they order drinks or start a conversation?
😎 Make a joke from the situation that I wrote above.
🤖 Why did the two people go into the pub? To get a pint to go with their feet!


## Labeling widget

In [ ]:
LABELING_DATASET = "tl" # "tl" for test-learn, "rq" for request-question

if LABELING_DATASET == "tl":
    print("Initiating labeling session for test-learn prompts")
    model_path = "reddgr/tl-test-learn-prompt-classifier"
    label_map = {0: "learn", 1: "test"}
    dataset_name = "reddgr/tl-test-learn-prompts"
elif LABELING_DATASET == "rq":
    print("Initiating labeling session for request-question prompts")
    model_path = "reddgr/rq-request-question-prompt-classifier"
    label_map = {0: "question", 1: "request"}
    dataset_name = "reddgr/rq-request-question-prompts"
else:
    raise ValueError(f"Invalid labeling dataset: {LABELING_DATASET}")

device = 0 if torch.cuda.is_available() else -1
classifier = pipeline("text-classification", model=model_path, tokenizer=model_path, device=device)
clear_output(wait=True)
prompt_labeling_widget = labeling_widget.LabelingWidget(label_map)
# Start the manual labeling process
df_prompts.rename(columns={'prompt': 'text'}, inplace=True)
prompt_labeling_widget.manual_labeling(df_prompts, classifier)

## Pushing labeled data to Hugging Face

In [11]:
prompt_labeling_widget.update_dataset(
    dataset_name=dataset_name,
    split_name="train", # Choose either test or train split
    hf_token=hf_token_write
)

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/15 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/2.98k [00:00<?, ?B/s]

c:\Users\david\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\david\.cache\huggingface\hub\datasets--reddgr--tl-test-learn-prompts. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Successfully pushed 15 records to reddgr/tl-test-learn-prompts train split.


________________________

## Pushing examples manually

Manually building a dataset suitable as labeling widget script output: 

In [4]:
# Create a list of dictionaries with label and text data. For example, we'll create some learn (0) and test (1) examples:
label_map = {0: "learn", 1: "test"}
dict_examples = [
    {'text': 'Who is David Mayer?', 'label': 0},
    {'text': 'Write David Mayer in leetspeak', 'label': 1},
    {'text': 'Why is ChatGPT not allowed to produce a response about David Mayer?', 'label': 0},
    {'text': 'Who👏is👏David👏Mayer👏de👏Rothschild?👏Answer👏clapping👏like👏this.👏', 'label': 1}
]
# Create a dataframe from the list of dictionaries
df_examples = pd.DataFrame(dict_examples)
display(df_examples)
new_dataset_records = Dataset.from_pandas(df_examples)
print(new_dataset_records)

,text,label
0,Who is David Mayer?,0
1,Write David Mayer in leetspeak,1
2,Why is ChatGPT not allowed to produce a respon...,0
3,Who👏is👏David👏Mayer👏de👏Rothschild?👏Answer👏clapp...,1


Dataset({
    features: ['text', 'label'],
    num_rows: 4
})


Pushing to hub:

In [5]:
dataset_name = "reddgr/tl-test-learn-prompts"
# Instantiate a labeling_widget object with the label map
manual_labeling_widget = labeling_widget.LabelingWidget(label_map)
# Push to Hugging Face hub directly by passing the dataframe with new examples to the update_dataset method
manual_labeling_widget.update_dataset(
    dataset_name=dataset_name,
    split_name="test", # Choose either test or train split
    hf_token=hf_token_write,
    new_dataset_records=new_dataset_records # The dataset we just created manually, without using the widget
)

Generating train split:   0%|          | 0/293 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/260 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/2.98k [00:00<?, ?B/s]

c:\Users\david\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\david\.cache\huggingface\hub\datasets--reddgr--tl-test-learn-prompts. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Successfully pushed 2 records to reddgr/tl-test-learn-prompts test split.
